# Invalid estimate diagnosis

The purpose of this notebook is to aid in diagnosing the cause of invalid precipitation estimates in the combined (final) data.

**expected**: for any duration $i$ and return interval $j$, the precipitation estimate at either $i,j + 1$ or $i + 1, j$ should be greater than the precipitation estimate at $i,j$. I.e., there should be a monotonic increasing relationship between precipitation estimate and both duration and return interval.

**issue**: This is not the case. Precipitation stimates for arbitrary durations and return intervals are less than estimates for the next largest duration at same return interval. Have not yet seen issue within a duration and increasing return interals however, appears to be limited to fixed return interval.  

## example

An example of this issue can be seen in the bounds for the **7d duration**, **1000yr interval** for **2020-2049 GFDL-CM3** output at latitude **64.66** and longitude **-147.96**. The **estimate is 18.87**, and the estimate for the **4d duration** in the **same (1000yr) return interval** is 22.5. 

## where: deltas or warping?

Does this problem appear to be present before or after the warping step? 

### testing
1. verify that applying the unwarped deltas to the Atlas 14 values for this particular location recreates the invalid estimates. 

In [1]:
# setup
import os, time
import lmoments3 as lmom
import numpy as np
import pandas as pd
import rasterio as rio
import scikits.bootstrap as boot
import xarray as xr
from lmoments3 import distr
from multiprocessing import Pool
from pyproj import Transformer

# directories
data_dir = "/workspace/Shared/Tech_Projects/DOT/project_data"
wrf_dir = os.path.join(data_dir, "wrf_pcpt")

# WGS84 coordinates from example of invalid bounds
wgs84_coords = (-147.96, 64.66)
# WRF CRS
wrf_crs = '+units=m +proj=stere +lat_ts=64.0 +lon_0=-152.0 +lat_0=90.0 +x_0=0 +y_0=0 +a=6370000 +b=6370000'
transformer = Transformer.from_proj("EPSG:4326", wrf_crs, always_xy=True)
wrf_coords = transformer.transform(*wgs84_coords)

/home/UA/kmredilla/.localpython/lib/python3.6/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
# function to query 1000yr delta at transformed coordinates
def get_delta(fp, wrf_coords=wrf_coords):
    deltas_ds = xr.open_dataset(fp)
    deltas_sel = deltas_ds.sel(xc=wrf_coords[0], yc=wrf_coords[1], method="nearest")
    return deltas_sel["pf"].values[-1]

In [3]:
deltas_fp = os.path.join(wrf_dir, "deltas", "pcpt_GFDL-CM3_sum_wrf_{}d_2020-2049_deltas.nc")
deltas = {
    "4d": get_delta(deltas_fp.format("4")),
    "7d": get_delta(deltas_fp.format("7"))
}

In [4]:
# get row/col for a14 grid, define window for rasters
transformer = Transformer.from_crs(4326, 3338, always_xy=True)
a14_coords = transformer.transform(*wgs84_coords)
a14_pf = rio.open(os.path.join(data_dir, "NOAA_Atlas14", "raw", "warped", "ak1000yr04da_ams.tif"))
a14_rc = a14_pf.index(*a14_coords)
window = ((a14_rc[0], a14_rc[0] + 1), (a14_rc[1], a14_rc[1] + 1))

In [5]:
# function to query Atlas 14 estimates for each duration in example
def get_a14_pf(fp, window=window):
    a14_src = rio.open(fp)
    return a14_src.read(1, window=window)[0][0] / 1000

In [6]:
a14_fp = os.path.join(data_dir, "NOAA_Atlas14", "raw", "warped", "ak1000yr0{}da_ams.tif")
estimates = {
    "4d": get_a14_pf(a14_fp.format("4")),
    "7d": get_a14_pf(a14_fp.format("7"))
}

Check for invalid estimates after multiplying deltas with estimates.

In [7]:
print("4d combined estimate:", round(estimates["4d"] * deltas["4d"], 3))
print("7d combined estimate:", round(estimates["7d"] * deltas["7d"], 3))

4d combined estimate: 25.541
7d combined estimate: 16.876


This confirms that the problem is present after the deltas step. 

## Possible solutions

### 1. rolling window method for durations step

This method was causing some computational issues when initially tested on entire dataset. The goal here will be to explore this possibility using the data for only the example location, and check whether the issue was rectified.

1. Load the historical and future data for the point-of-interest

In [8]:
# get xc, yc indices for the WRF coordinates
wrf_fp = os.path.join(wrf_dir, "pcpt", "pcpt_hourly_wrf_{}_{}_{}.nc")
pcpt_ds = xr.open_dataset(wrf_fp.format("GFDL-CM3", "historical", "1970"))
xc = pcpt_ds.xc.values
yc = pcpt_ds.yc.values
idx = (np.abs(xc - wrf_coords[0]).argmin(), np.abs(yc - wrf_coords[1]).argmin())
pcpt_ds.close()

In [9]:
# function to query a WRF pcpt dataset for all times
def get_wrf_pcpt(fp, xc_idx=idx[0], yc_idx=idx[1]):
    ds = xr.open_dataset(fp)
    time = ds.time.values
    pcpt = ds["pcpt"].values[:,yc_idx,xc_idx]
    ds.close()
    da = xr.DataArray(pcpt, coords=[time], dims=["time"])
    return da

In [10]:
# read historical data
tic = time.perf_counter()

years = np.arange(1970, 2006).astype("str")
wrf_fps = [wrf_fp.format("GFDL-CM3", "historical", year) for year in years]
p = Pool(24)
hist_da_list = p.map(get_wrf_pcpt, wrf_fps)
p.close()

print(round(time.perf_counter() - tic, 1), "s")

51.6 s


In [11]:
# read future data
tic = time.perf_counter()

years = np.arange(2020, 2050).astype("str")
wrf_fps = [wrf_fp.format("GFDL-CM3", "rcp85", year) for year in years]
p = Pool(24)
rcp85_da_list = p.map(get_wrf_pcpt, wrf_fps)
p.close()

print(round(time.perf_counter() - tic, 1), "s")

52.7 s


In [12]:
# concat data arrays
hist = xr.concat(hist_da_list, "time")
rcp85 = xr.concat(rcp85_da_list, "time")

2. calculate the durations series - sum the precip amounts over 4 day and 7 day periods, implementing both rolling and binning methods

In [13]:
# resample to the respective durations
def calc_dur(da):
    dur_di = {
        "4d": {
            "bin": da.resample(time="4D").sum(), 
            "roll": da.rolling(time=(4 * 24)).sum().dropna("time")
        },
        "7d": {
            "bin": da.resample(time="7D").sum(), 
            "roll": da.rolling(time=(7 * 24)).sum().dropna("time")
        },
    }
    return dur_di

hist_dur = calc_dur(hist)
rcp85_dur = calc_dur(rcp85)

verify that the calculated durations series match the durations series computed from the pipeline run

In [14]:
# open durations file, query at POI
def get_dur(fp, xc=wrf_coords[0], yc=wrf_coords[1]):
    dur_ds = xr.open_dataset(fp)
    return dur_ds.sel(xc=xc, yc=yc, method="nearest").pcpt

dur_fp = os.path.join(wrf_dir, "durations", "pcpt_{}_sum_wrf_{}_{}.nc")
pipe_dur = {
    "4d": {
        "hist": get_dur(dur_fp.format("4d", "GFDL-CM3", "historical")), 
        "rcp85": get_dur(dur_fp.format("4d", "GFDL-CM3", "rcp85"))
    }, 
    "7d": {
        "hist": get_dur(dur_fp.format("7d", "GFDL-CM3", "historical")), 
        "rcp85": get_dur(dur_fp.format("7d", "GFDL-CM3", "rcp85"))
    }
}

print("Sample of precip sums from pipeline:", pipe_dur["4d"]["hist"].values[:5])
print("Sample of precip sums calculated from raw WRF:", hist_dur["4d"]["bin"].values[:5])

Sample of precip sums from pipeline: [ 3.782     12.1015     5.26      17.661499   0.9404998]
Sample of precip sums calculated from raw WRF: [ 3.278      7.8299994 10.0355     6.111     11.969    ]


They don't match. This warrants further investigation.

However, this mthod can still be tested with the manually computed durations series.

3. Calculate annual maximum series using both rolling and original methods (for both sets of durations)

In [15]:
# function to return ditcionary of ams from dict of durations
def run_calc_ams(dur_di, yr_sl=("1979", "2015")):
    def calc_ams(di):
        return {k: da.sel(time=slice(yr_sl[0], yr_sl[1])).resample(time="Y").max() for k, da in di.items()}
    return {k: calc_ams(sub_di) for k, sub_di in dur_di.items()}

hist_ams = run_calc_ams(hist_dur)
rcp85_ams = run_calc_ams(rcp85_dur, ("2020", "2049"))

4. calculate deltas

In [16]:
def fit_values(dat):
    paras = distr.gev.lmom_fit(dat)
    return distr.gev(**paras)

# compute deltas from global dicts
def calc_delta(hist, rcp85):
    p = 1 - 1/1000
    hist_fit = fit_values(hist)
    rcp_fit = fit_values(rcp85)
    return round(rcp_fit.ppf(p) / hist_fit.ppf(p), 3)

def run_calc_delta(dur, sub_di):
    return {k: calc_delta(da.values, rcp85_ams[dur][k]) for k, da in sub_di.items()}

deltas_di = {k: run_calc_delta(k, sub_di) for k, sub_di in hist_ams.items()}

In [17]:
deltas_di

{'4d': {'bin': 4.587, 'roll': 4.708}, '7d': {'bin': 5.045, 'roll': 3.559}}

The 4d delta actually increases, whereas the 7d delta decreases. This would result in a greater inconsistency. 

This rules out the rolling window method for the durations step as a potential solution. 

Curious about whether the ordering in the output intervals step is preserved, however.

Here are the historical estimates based on durations method:

In [18]:
p = 1 - 1/1000
colnames = ["Binning", "Rolling"]
methods = ["bin", "roll"]

def get_df(ams_di):
    return pd.DataFrame.from_dict(
        {
            k: [
                round(fit_values(sub_di[method]).ppf(p), 2) for method, da in sub_di.items()
            ] for k, sub_di in ams_di.items()
        },
        orient="index", columns=colnames
    )
    

hist_pf = get_df(hist_ams)
rcp85_pf = get_df(rcp85_ams)

In [19]:
hist_pf

,Binning,Rolling
4d,73.54,66.09
7d,71.61,101.86


and the future estimates for each method:

In [20]:
rcp85_pf

,Binning,Rolling
4d,337.35,311.14
7d,361.30,362.56


and the resulting deltas:

In [21]:
deltas_df = np.round(rcp85_pf / hist_pf, 2)
deltas_df

,Binning,Rolling
4d,4.59,4.71
7d,5.05,3.56


Multiplying each of these by the corresponding 4d and 7d Atlas 14 1000yr estimates of 7.18 and 8.18, respectively:

In [22]:
a14_lu = {"4d": 7.18, "7d": 8.18}
combined = deltas_df.copy()
for idx in deltas_df.index.values:
    combined.loc[idx] = np.round(deltas_df.loc[idx] * a14_lu[idx], 2)

In [23]:
combined

,Binning,Rolling
4d,32.96,33.82
7d,41.31,29.12
